In [264]:
import nbimporter
from typing import Dict, Tuple, List
import numpy as np

### Env Vars

In [361]:
QUBITS_NUM = 4 
N = 16
K = 4
NUM_SHOTS = 10000

### Simulator Backend

In [253]:
from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals

seed = 50
algorithm_globals.random_seed = seed

simulator_backend = Aer.get_backend('qasm_simulator')

### BFGS Optimizer

In [35]:
from qiskit.algorithms.optimizers import L_BFGS_B

bfgs_optimizer = L_BFGS_B(maxiter=60)

### k input states (computational basis)

In [36]:
from utiles import *

In [37]:
k = 4
n = 16
input_states = get_first_k_eigenvectors_from_n_computational_basis(k, n)
print(input_states)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


### Ansatz State

In [261]:
from ansatz_circuit_item2 import get_full_variational_quantum_circuit

In [262]:
init_circuit_params = {
    "thetas": np.random.uniform(low=0, high=2*np.pi, size=8),
    "phis": np.random.uniform(low=0, high=2*np.pi, size=4),
    "D1": 2,
    "D2": 6
}

In [265]:
def prepare_circuit_params(thetas) -> Dict:
     return {
    "thetas": thetas[4:],
    "phis": thetas[:4],
    "D1": 2,
    "D2": 6
     }

In [197]:
def get_ansatz_state(circuit_params, input_state):
    circuit_params_with_input_state = {**circuit_params, "input_state": input_state}
    return get_full_variational_quantum_circuit(**circuit_params_with_input_state)

## Expectation Value

### From hamiltonian to pauli string

In [380]:
from qiskit.quantum_info import Pauli
S_t = S.adjoint()
X_z = H@Z@H
Y_z = S@H@Z@H@S_t

reducing_to_pauli_z_dict = {
    Pauli('I'): Pauli('I'),
    Pauli('Z'): Pauli('Z'),
    Pauli('X'): Pauli('HZH'),
    Pauli('Y'): Pauli('SHZHS')
} 

QiskitError: 'Pauli string label "HZH" is not valid.'

In [362]:
#TODO: change the a_i and J_ij params into random values
from qiskit.opflow import X, Z, Y, I, H, S
from qiskit.opflow import ListOp

H_transverse_ising = 0.5*((I^I^I^X) + (I^I^X^I) + (I^X^I^I) + (X^I^I^I) + \
                          (Z^Z^I^I) + (Z^I^Z^I) + (Z^I^I^Z) + (I^Z^Z^I) + \
                          (I^Z^I^Z) + (I^I^Z^Z))

In [359]:
def transfrom_hamiltonian_into_pauli_string(hamiltonian):
    pauli_operators = hamiltonian.to_pauli_op().settings['oplist']
    pauli_strings = list(map(lambda pauli_operator: pauli_operator.primitive, pauli_operators))
    return pauli_strings

In [377]:
def reduce_pauli_matrixes_into_sigma_z(pauli_string):
    for matrix_index in range(QUBITS_NUM):
        pauli_matrix = pauli_string[matrix_index]
        pauli_string[matrix_index].insert(reducing_to_pauli_z_dict[pauli_matrix.primitive_strings])
    
    print(pauli_string)
    return pauli_string

In [371]:
NUM_SHOTS=10000

def get_probability_distribution(counts: Dict) -> Dict:
    proba_distribution = {state: (count / NUM_SHOTS) for state, count in counts.items()}
    print(proba_distribution)
    return proba_distribution

def calculate_probabilities_of_measurments_in_computational_basis(quantum_state_circuit):
    quantum_state_circuit.measure_all()
    
    transpiled_quantum_state_circuit = transpile(quantum_state_circuit, simulator_backend)
    Qobj = assemble(transpiled_quantum_state_circuit)
    result = simulator_backend.run(Qobj).result()
    counts = result.get_counts(quantum_state_circuit)
    print(counts)
    
    return get_probability_distribution(counts)


In [372]:
def get_expectation_value(hamiltonian, ansatz_state):
    proba_distribution = calculate_probabilities_of_measurments_in_computational_basis(ansatz_state)
    
    pauli_strings = transfrom_hamiltonian_into_pauli_string(hamiltonian)
    pauli_strings_reduced_to_sigma_z =  list(map(lambda pauli_string: 
                                                 reduce_pauli_matrixes_into_sigma_z(pauli_string), pauli_strings))
    
    right_mult = np.matmul(hamiltonian, np.array(ket_state_vector))
    bra_state_vector = np.matrix(ket_state_vector).getH()
    return np.matmul(right_mult, bra_state_vector)[(0,0)]

## Objective Function

In [373]:
from qiskit import assemble, transpile
k = 4
n = 16
w = 0.5

def get_cost_function(k, thetas, hamiltonian):    
    circuit_params = prepare_circuit_params(thetas)
    computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(k, n)
    
    ansatz_state = get_ansatz_state(circuit_params, computational_eigenvectors[k-1])
    L_w = w*get_expectation_value(hamiltonian, ansatz_state)
    
    for j in range(k-1):
        ansatz_state = get_ansatz_state(circuit_params, computational_eigenvectors[j])
        L_w += get_expectation_value(hamiltonian, ansatz_circuit)
        
    return L_w

In [378]:
thetas = np.random.uniform(low=0, high=2*np.pi, size=12)
get_cost_function(k=4, thetas=thetas, hamiltonian=H_transverse_ising)

{'0111': 440, '0011': 99, '0101': 152, '1111': 53, '1011': 170, '0001': 11, '1001': 17, '1101': 20, '0100': 37, '0110': 9, '1100': 6, '0000': 6, '1000': 1, '1110': 1, '0010': 1, '1010': 1}
{'0111': 0.044, '0011': 0.0099, '0101': 0.0152, '1111': 0.0053, '1011': 0.017, '0001': 0.0011, '1001': 0.0017, '1101': 0.002, '0100': 0.0037, '0110': 0.0009, '1100': 0.0006, '0000': 0.0006, '1000': 0.0001, '1110': 0.0001, '0010': 0.0001, '1010': 0.0001}


AttributeError: 'Pauli' object has no attribute 'primitive_strings'